In [12]:
%matplotlib inline
from __future__ import division
import matplotlib.pyplot as plt
from astropy.io import fits
import numpy as np
from astropy.cosmology import FlatLambdaCDM
from astropy import units as u
from astropy.table import Table, hstack, vstack
from scipy.stats import ks_2samp, beta
from functools import reduce

group_table = Table(fits.getdata('../group_table2.fits'))
galaxy_table = Table(fits.getdata('../galaxy_table2.fits'))

fl = fits.open('../JHU_mstell_ssfr_Yang_gzoo_renamed.fits')
alldata = Table(fl[1].data)

In [13]:
alldata.info

<Table length=232014>
                             name                              dtype  shape
------------------------------------------------------------- ------- -----
                                                MEDIAN_MSTELL float32      
                                                        P16_1 float32      
                                                        P84_1 float32      
                                                       P2P5_1 float32      
                                                      P97P5_1 float32      
                                                       MODE_1 float32      
                                                       mstell float32      
                                                      PLATEID   int16      
                                                          MJD   int32      
                                                      FIBERID   int16      
                                                      PHOTOID   in

In [17]:
#this cell gets IDs of smooth, starforming centrals

group_ind = np.where(group_table['n_gal'] > 2)[0]
halo_mass_val_ind = np.where(group_table['halo_mass_lum'] > 2)[0]
sf_cent_ind = np.where(group_table['sfing_central'] == 'SFing')[0]
smooth_half_cent_ind = np.where(group_table['cent_pfeat'] < 0.5)[0]

sfing_smooth_cent_group_ind = reduce(np.intersect1d, (halo_mass_val_ind, group_ind, sf_cent_ind, smooth_half_cent_ind))

sfing_smooth_cent_group_nums = group_table['group_num'][sfing_smooth_cent_group_ind]
inds = []

print(sfing_smooth_cent_group_nums)
for group_num in sfing_smooth_cent_group_nums:
    #print(group_num)
    gals = np.where(alldata['group_num'] == group_num)[0]
    for gal in gals:
        if alldata['centsat'][gal] == 1:
            inds.append(gal)

sfsc = alldata[inds]
       
short_sfing_smooth_cents = Table([sfsc['PLATEID'], sfsc['MJD'], sfsc['FIBERID'], sfsc['RA_1'], sfsc['DEC_1'], sfsc['z'], sfsc['ssfr'], sfsc['NYU-VAGC_ID'], sfsc['M_r-5logh'], sfsc['galaxy_num'], sfsc['group_num'], sfsc['centsat'], sfsc['dr7objid'], sfsc['pfeatures'], sfsc['mstell']])

group_num
---------
     2419
     2579
     3102
     3731
     4705
     5993
     6010
     7470
     9799
    10001
    11535
    14213
    15750
    16523
    18311
    18577
    19543
    22561


In [23]:
short_sfing_smooth_cents

PLATEID,MJD,FIBERID,RA_1,DEC_1,z,ssfr,NYU-VAGC_ID,M_r-5logh,galaxy_num,group_num,centsat,dr7objid,pfeatures,mstell,description
int16,int32,int16,float32,float32,float32,float32,int64,float64,int64,int64,int64,int64,float32,float32,str38
1877,54464,121,140.10027,65.44682,0.037898827,-10.25883,1784454,-20.9398918,433611,2419,1,587738195585204296,0.333,11.076331,red elliptical
327,52294,440,173.18034,-2.199732,0.04165886,-10.178249,167241,-20.8882217,20727,2579,1,587724649790898308,0.45,10.826029,"reddish blue, has a spiral arm"
480,51989,321,145.61258,2.452593,0.04901921,-10.52835,287734,-20.6466465,60647,3102,1,587726033307238594,0.251,10.713894,"reddish, maybe lenticular"
326,52375,461,171.66945,-1.6937925,0.0464054,-10.18143,168692,-21.2200394,21098,3731,1,587724650327113796,0.146,10.72525,"bluer, circular, bright center"
327,52294,112,174.08037,-2.8474183,0.045133978,-10.31015,173987,-20.8426666,23070,4705,1,587725040089301073,0.486,10.848811,"bluish, lenticular, maybe spiral arms"
2754,54240,527,233.30078,12.910944,0.052541777,-9.59732,2368385,-20.0084629,608942,5993,1,587742575925919987,0.201,10.717148,reddish lenticular (?)
2480,53851,267,160.76328,15.561178,0.05051067,-10.30071,2405762,-20.9587574,621135,6010,1,587742863667626206,0.106,10.871525,"bluish elliptical, bright center"
1419,53144,47,243.06876,32.108803,0.05852842,-10.20811,1413112,-20.736824,393628,7470,1,587736782533820553,0.083,10.722824,"merger?, bluer, some structure?"
880,52367,636,176.92368,52.44885,0.04823265,-10.42148,872384,-21.1768017,198201,9799,1,587732135921057870,0.149,10.812687,"bluer elliptical, maybe lenticular"


In [22]:
#descriptions of each object from sdss skyserver quick look
desc = ['red elliptical', 'reddish blue, has a spiral arm', 'reddish, maybe lenticular', 'bluer, circular, bright center', 'bluish, lenticular, maybe spiral arms', 'reddish lenticular (?)', 'bluish elliptical, bright center', 'merger?, bluer, some structure?', 'bluer elliptical, maybe lenticular', 'reddish elliptical', 'bluish, spiral arms(?)', 'reddish lenticular', 'bluish, face on spiral', 'lenticular?', 'bluish elliptical', 'reddish elliptical, some structure (?)', 'merger, bluish maybe spiral arms', 'bluish lenticularish']
short_sfing_smooth_cents['description'] = desc


short_sfing_smooth_cents.write('../sfing_smooth_cents_table.fits', format='fits', overwrite=True)